In [1]:
import sys
sys.path.append('../')
from Code.envs.statemachine import SuccessiveLookups
from Code.train import train, OptWrapper
from Code.everything5 import build_standard_loop
import torch
from PIL import Image
import numpy as np
import torch.nn as nn

In [2]:
MAIN_DECAY = np.exp(-1/(20)*0.5)
ADAP_DECAY = np.exp(-1/(20*2))

spec = {
    'control_config': {
        'neuron_type': 'LIF',
        'n_neurons': 100,
        'BETA': 0.8,
        '1-beta': 'improved',
        'SPIKE_FN': 'bellec'
    },
    'mem_config': {
        'neuron_type': 'FlipFlop',
        'n_neurons': 20,
        'BETA': 0.8,
        '1-beta': 'improved',
        'SPIKE_FN': 'bellec',
        'ADAPSCALE': 30,
        'ADAPDECAY': None, #TODO: set this
        'OFFSET': None,
        'DECAY': MAIN_DECAY
    },
    'exp_config': {
        'n_sequence': 30,
        'val_sequence': 100,
        'round_length': 20
    },
    'lr': 0.001,
    'lr_decay': 1,
    'iterations': 5000,
    'batch_size': 64,
    'architecture': '1L'
}

In [3]:
DEVICE = torch.device('cuda')


train_problem = SuccessiveLookups(spec['iterations'], spec['batch_size'], spec['exp_config']['n_sequence'],
                               spec['exp_config']['round_length'], DEVICE)
val_problem = SuccessiveLookups(1, spec['batch_size'], spec['exp_config']['val_sequence'],
                               spec['exp_config']['round_length'], DEVICE)


n_in, n_out, input_rate = train_problem.get_infos()




In [4]:
from Code.everything5 import OuterWrapper, DynNetwork, ParallelNetwork2, SequenceWrapper, BaseNeuron

loop = build_standard_loop(spec, n_in, input_rate)
out_neuron_size = spec['control_config']['n_neurons'] + spec['mem_config']['n_neurons']

outer = {
    'input': n_in,
    'loop': [['input'], SequenceWrapper(ParallelNetwork2(loop)), None],
    'output': [['loop'], BaseNeuron(n_out, None), nn.Linear]
}

model = OuterWrapper(DynNetwork(outer))
model.to(DEVICE)

{'control': [[('input', 1), ('control', 1), ('mem', 1)], LIFNeuron(), <class 'torch.nn.modules.linear.Linear'>], 'mem': [[('input', 1), ('control', 1), ('mem', 1)], NewFlipFlop(
  (lif_on): LIFNeuron()
  (lif_off): LIFNeuron()
), <class 'torch.nn.modules.linear.Linear'>], 'output': [[('control', 1), ('mem', 1)], BaseNeuron(), None]}


OuterWrapper(
  (model): DynNetwork(
    (layers): ModuleDict(
      (loop): SequenceWrapper(
        (model): ParallelNetwork2(
          (layers): ModuleDict(
            (control): LIFNeuron()
            (control_synapse): Linear(in_features=148, out_features=100, bias=True)
            (mem): NewFlipFlop(
              (lif_on): LIFNeuron()
              (lif_off): LIFNeuron()
            )
            (mem_synapse): Linear(in_features=148, out_features=20, bias=True)
            (output): BaseNeuron()
          )
        )
      )
      (output_synapse): Linear(in_features=120, out_features=8, bias=True)
      (output): BaseNeuron()
    )
  )
)

In [5]:
optimizer = OptWrapper(model.parameters(), spec['lr'], spec['lr_decay'], 2500)

In [ ]:
train(train_problem, val_problem, optimizer, model, None)

Val Acc: 13.02% | Val Time: 2.4s | Time per it: 2.3s
It:   20 | Loss: 2.084 | Acc: 12.37%
It:   40 | Loss: 2.077 | Acc: 14.73%
It:   60 | Loss: 2.064 | Acc: 15.23%
It:   80 | Loss: 2.060 | Acc: 14.52%
It:  100 | Loss: 2.068 | Acc: 14.57%
Val Acc: 13.34% | Val Time: 2.6s | Time per it: 2.2s
It:  120 | Loss: 2.040 | Acc: 16.54%
It:  140 | Loss: 2.054 | Acc: 15.11%
It:  160 | Loss: 2.024 | Acc: 19.24%
It:  180 | Loss: 1.992 | Acc: 22.69%
It:  200 | Loss: 1.939 | Acc: 25.63%
Val Acc: 24.95% | Val Time: 2.4s | Time per it: 2.1s
It:  220 | Loss: 2.011 | Acc: 22.43%
It:  240 | Loss: 2.023 | Acc: 21.25%
It:  260 | Loss: 1.958 | Acc: 22.34%
It:  280 | Loss: 1.900 | Acc: 27.21%
It:  300 | Loss: 1.876 | Acc: 26.82%
Val Acc: 27.73% | Val Time: 2.5s | Time per it: 2.2s
It:  320 | Loss: 1.954 | Acc: 25.81%
It:  340 | Loss: 1.934 | Acc: 25.06%
It:  360 | Loss: 1.910 | Acc: 24.95%
It:  380 | Loss: 1.879 | Acc: 26.96%
It:  400 | Loss: 1.814 | Acc: 33.00%
Val Acc: 33.65% | Val Time: 2.5s | Time per it: 